In [ ]:
import pandas as pd
import requests
from datetime import datetime, timedelta

# Define API endpoints and parameters
octopus_energy_api_url = 'https://api.octopus.energy/v1/products/'
carbon_intensity_api_url = 'https://api.carbonintensity.org.uk/intensity/date'
google_api_url = 'https://maps.googleapis.com/maps/api/'

# Define the date range for the data
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 1, 7)

# Initialize empty lists to store data
timestamps = []
energy_consumption = []
renewable_generation = []
storage_capacity = []
storage_charge_rate = []
storage_discharge_rate = []
carbon_intensity = []

# Generate a date range with hourly intervals
date_range = pd.date_range(start=start_date, end=end_date, freq='H')

# Example function to fetch data from the Carbon Intensity API
def fetch_carbon_intensity(date):
    response = requests.get(f"{carbon_intensity_api_url}/{date.strftime('%Y-%m-%d')}")
    data = response.json()
    return data['data']

# Example function to fetch data from the Octopus Energy API
def fetch_octopus_energy_data():
    # This function should include API key and actual data fetching logic
    pass

# Example function to fetch Google API data
def fetch_google_api_data():
    # This function should include API key and actual data fetching logic
    pass

# Populate the lists with synthetic or fetched data
for timestamp in date_range:
    timestamps.append(timestamp)
    energy_consumption.append(100 + (50 * timestamp.hour % 24))  # Synthetic data example
    renewable_generation.append(200 + (30 * timestamp.hour % 24))  # Synthetic data example
    storage_capacity.append(500)  # Synthetic constant value
    storage_charge_rate.append(25)  # Synthetic constant value
    storage_discharge_rate.append(25)  # Synthetic constant value
    carbon_intensity.append(fetch_carbon_intensity(timestamp))  # Fetch carbon intensity data

# Create a DataFrame
data = pd.DataFrame({
    'timestamp': timestamps,
    'energy_consumption': energy_consumption,
    'renewable_generation': renewable_generation,
    'storage_capacity': storage_capacity,
    'storage_charge_rate': storage_charge_rate,
    'storage_discharge_rate': storage_discharge_rate,
    'carbon_intensity': carbon_intensity
})

# Save to CSV
data.to_csv('energy_data.csv', index=False)

print("Data has been saved to energy_data.csv")


Data has been saved to energy_data.csv


In [ ]:
df = pd.read_csv('energy_data.csv')

print(df.to_string())

               timestamp  energy_consumption  renewable_generation  storage_capacity  storage_charge_rate  storage_discharge_rate                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data (example for demonstration purposes)
data = pd.read_csv('energy_data.csv')

# Preprocess data
# Example preprocessing steps
data.fillna(method='ffill', inplace=True)
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp', inplace=True)

# Feature engineering
data['hour'] = data.index.hour
data['day'] = data.index.dayofweek

# Split data
X = data.drop('carbon_intensity', axis=1)
y = data['carbon_intensity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
!pip install scikeras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from scikeras.wrappers import KerasRegressor

# Define LSTM model
def create_lstm_model():
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Define CNN model
def create_cnn_model():
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Define DNN model
def create_dnn_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model


In [ ]:
from sklearn.ensemble import VotingRegressor
from scikeras.wrappers import KerasRegressor
from sklearn.metrics import mean_absolute_error

# Wrapping models for ensemble
lstm_model = KerasRegressor(build_fn=create_lstm_model, epochs=50, batch_size=32, verbose=0)
cnn_model = KerasRegressor(build_fn=create_cnn_model, epochs=50, batch_size=32, verbose=0)
dnn_model = KerasRegressor(build_fn=create_dnn_model, epochs=50, batch_size=32, verbose=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ensemble Model
ensemble = VotingRegressor(estimators=[('lstm', lstm_model), ('cnn', cnn_model), ('dnn', dnn_model)])
ensemble.fit(X_train, y_train)

# Evaluation
y_pred = ensemble.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')


ValueError: could not convert string to float: "[{'from': '2023-01-01T00:00Z', 'to': '2023-01-01T00:30Z', 'intensity': {'forecast': 73, 'actual': 72, 'index': 'low'}}, {'from': '2023-01-01T00:30Z', 'to': '2023-01-01T01:00Z', 'intensity': {'forecast': 63, 'actual': 80, 'index': 'low'}}, {'from': '2023-01-01T01:00Z', 'to': '2023-01-01T01:30Z', 'intensity': {'forecast': 71, 'actual': 72, 'index': 'low'}}, {'from': '2023-01-01T01:30Z', 'to': '2023-01-01T02:00Z', 'intensity': {'forecast': 76, 'actual': 65, 'index': 'low'}}, {'from': '2023-01-01T02:00Z', 'to': '2023-01-01T02:30Z', 'intensity': {'forecast': 72, 'actual': 65, 'index': 'low'}}, {'from': '2023-01-01T02:30Z', 'to': '2023-01-01T03:00Z', 'intensity': {'forecast': 66, 'actual': 65, 'index': 'low'}}, {'from': '2023-01-01T03:00Z', 'to': '2023-01-01T03:30Z', 'intensity': {'forecast': 67, 'actual': 67, 'index': 'low'}}, {'from': '2023-01-01T03:30Z', 'to': '2023-01-01T04:00Z', 'intensity': {'forecast': 67, 'actual': 68, 'index': 'low'}},